<a href="https://colab.research.google.com/github/jasperSha/cloud_music/blob/main/feature_collection/mel_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud projects list

In [2]:
import os

os.makedirs('song_images', exist_ok=True)

os.makedirs('song_flacs', exist_ok=True)

In [ ]:
# load audio every N batches
# have to manually enter (only have to change tail input to starting index each time, head just adds to it)
start = 0
end = 500

'''
TODO: for each batch, create csv of just the id's of song images created, use that for reference(instead of loading the pngs)
'''
# load image records data
!gsutil -m cp gs://deepclustermusic/song_images/*.csv .

!gsutil ls gs://deepclustermusic/flac_files/spotify_yt_data/*.flac | tail -n +0 | head -n 500 | gsutil -m cp -n -I './song_flacs'


In [ ]:
!pip install fastaudio &> /dev/null

In [5]:
import glob

import numpy as np
import pandas as pd

import librosa
import librosa.feature
import librosa.display

import skimage.io

import torchaudio.transforms as ta
import torch
import torchaudio

from IPython.display import Audio, display
import matplotlib.pyplot as plt

In [123]:
def norm_wav(sample):
  return (sample - sample.mean()) / sample.std()

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled


In [162]:
mel_config = dict(
    sr=44100,
    hop_length=441,
    win_length=1024, # 1024 combined with n_fft=4096 best; 2205 -> 50ms, 4410 -> 100ms
    n_mels=256,
    n_fft=4096,
)

# only first 10 seconds up to 2 minutes of each
start_time = mel_config['sr'] * 10
end_time = mel_config['sr'] * 120

In [117]:
flacs = list(glob.glob('/content/song_flacs/*.flac'))

fin = list(glob.glob('*.csv'))
finished_df = pd.concat((pd.read_csv(f) for f in fin))

In [ ]:
songs = []
for fname in flacs:
  song_id = fname.split('/')[-1].split('.')[0]
  # if song_id in finished_df['song_id'].values:
    print('already finished this one')
    continue
  songs.append(song_id)

  signal, _ = torchaudio.load(fname)
  if signal is None:
    continue

  # convert any stereo to mono
  channel = signal.ndim
  if channel == 2:
    signal = torch.mean(signal, dim=0, keepdim=True)

  # get duration in seconds
  num_elements = torch.numel(signal)
  sr = mel_config['sr']
  duration = int(num_elements/sr)

  # set minimum duration
  if duration < 100 or duration > 480:
    continue

  signal = signal.T
  signal = signal[:,0]
  signal = signal.numpy()

  # normalize the signal
  signal = norm_wav(signal)
  window = signal[start_time:end_time]
  mels = librosa.feature.melspectrogram(window, **mel_config)
  img = librosa.power_to_db(mels, ref=np.max)

  img = plt.imshow(img, aspect='auto')
  plt.axis('off')
  plt.savefig("./song_images/%s_melspec.png"%song_id, bbox_inches='tight', pad_inches=0)

  plt.clf()
  plt.close('all')



In [166]:
df = pd.DataFrame(songs, columns=['song_id'])
df.to_csv('%s_%s.csv'%(start, end), encoding='utf-8', index=False)

In [ ]:
!gsutil -m cp -n song_images/*.png gs://deepclustermusic/song_images/
!gsutil cp 0_500.csv gs://deepclustermusic/song_images/